In [ ]:
import numpy as np
class GaussianNaiveBayes:
    """
    A Naive Bayes classifier that models all features as Gaussian
    (continuous numerical values following a normal distribution).

    All probability calculations are done in log space to avoid numerical
    underflow when probabilities become very small. During prediction the model
    converts log probabilities back into normal probabilities and normalizes
    them so that each row sums to one.

    Feature likelihoods
    -------------------
    For Gaussian numeric features:
        log_pdf = -0.5 * ( log(2 * pi * variance) + ((x - mean)^2) / variance )

    The model uses maximum likelihood estimates of the mean and variance
    for each feature within each class, with an additional variance smoothing
    term added for numerical stability.

    Parameters
    ----------
    eps : float
        Small constant added to Gaussian variances for stability.

    Notes
    -----
    This version of the classifier supports only continuous numerical features.
    Any preprocessing or transformations needed to produce numeric inputs
    must be done before calling fit.

    Example
    -------
    >>> X = np.array([[5.1, 3.5],
                      [4.9, 3.0],
                      [5.0, 3.4]])
    >>> y = np.array(["A", "B", "A"])
    >>> model = MixedNaiveBayes()
    >>> model.fit(X, y)
    >>> model.predict(X)
    array(['A', 'B', 'A'], dtype='<U1')
    """

    def __init__(self, eps=1e-9):
        self.eps = eps

        self.feature_types = None
        self.classes_ = None

        self.log_priors_ = {}

        self.gaussian_means_ = {}
        self.gaussian_vars_ = {}
        self.bernoulli_probs_ = {}
        self.categorical_probs_ = {}
        self.categorical_values_ = {}

    # Likelihood helper functions
    def _gaussian_log_pdf(self, x, mean, var):
        """
        Computes the log density of a Gaussian distribution for a single feature.
        This function is for continuous values.
        """
        return -0.5 * (np.log(2.0 * np.pi * var) + ((x - mean) ** 2) / var)

    # Aggregator
    def _compute_log_likelihood(self, x_row, class_label):
        """
        Computes the total log likelihood of a sample under a specific class.
        This is the sum of the log likelihoods of each feature.
        """
        log_l = 0.0

        for j in range(len(x_row)):
            mean = self.gaussian_means_[class_label][j]
            var  = self.gaussian_vars_[class_label][j]
            log_l += self._gaussian_log_pdf(x_row[j], mean, var)

        return log_l

    # Fit
    def fit(self, X, y):
        """
        Learns Gaussian Naive Bayes parameters: class priors,
        per-class means and variances (with sklearn-style smoothing).

        Inputs
        ------
        X : array-like of shape (n_samples, n_features)
            Training data (numeric only).
        y : array-like of shape (n_samples,)
            Class labels.

        Output
        ------
        Returns the trained model instance.
        """
        X = np.asarray(X, float)
        y = np.asarray(y)

        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)

        # global variance for smoothing
        global_var = X.var()

        # class priors
        for c in self.classes_:
            n_c = np.sum(y == c)
            self.log_priors_[c] = np.log(n_c / n_samples)

        # init Gaussian dicts
        for c in self.classes_:
            self.gaussian_means_[c] = {}
            self.gaussian_vars_[c] = {}

        # compute Gaussian parameters
        for c in self.classes_:
            Xc = X[y == c]

            means = Xc.mean(axis=0)
            vars_  = Xc.var(axis=0)

            # smoothing:
            # var += eps * global_variance
            smoothed_var = vars_ + self.eps * global_var

            for j in range(n_features):
                self.gaussian_means_[c][j] = means[j]
                self.gaussian_vars_[c][j] = smoothed_var[j]

        return self

    # Prediction
    def predict_log_proba(self, X):
        """
        Computes unnormalized log posterior scores for each class.
        These are the raw log scores before normalization.
        """
        X = np.asarray(X, float)
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, len(self.classes_)))

        for i in range(n_samples):
            x_row = X[i]
            for k, c in enumerate(self.classes_):
                log_l = self._compute_log_likelihood(x_row, c)
                log_probs[i, k] = self.log_priors_[c] + log_l

        return log_probs

    def predict_proba(self, X):
        """
        Converts log posterior scores into normalized probabilities.
        Uses the log-sum-exp trick for numerical stability.
        """
        log_probs = self.predict_log_proba(X)

        max_log = np.max(log_probs, axis=1, keepdims=True)
        shifted = log_probs - max_log

        probs = np.exp(shifted)
        probs = probs / probs.sum(axis=1, keepdims=True)

        return probs

    def predict(self, X):
        """
        Returns the predicted class label for each sample.
        """
        probs = self.predict_proba(X)
        class_indices = np.argmax(probs, axis=1)
        return self.classes_[class_indices]